# Assignment 4

Before working on this assignment please read these instructions fully. In the submission area, you will notice that you can click the link to **Preview the Grading** for each step of the assignment. This is the criteria that will be used for peer grading. Please familiarize yourself with the criteria before beginning the assignment.

This assignment requires that you to find **at least** two datasets on the web which are related, and that you visualize these datasets to answer a question with the broad topic of **weather phenomena** (see below) for the region of **Ann Arbor, Michigan, United States**, or **United States** more broadly.

You can merge these datasets with data from different regions if you like! For instance, you might want to compare **Ann Arbor, Michigan, United States** to Ann Arbor, USA. In that case at least one source file must be about **Ann Arbor, Michigan, United States**.

You are welcome to choose datasets at your discretion, but keep in mind **they will be shared with your peers**, so choose appropriate datasets. Sensitive, confidential, illicit, and proprietary materials are not good choices for datasets for this assignment. You are welcome to upload datasets of your own as well, and link to them using a third party repository such as github, bitbucket, pastebin, etc. Please be aware of the Coursera terms of service with respect to intellectual property.

Also, you are welcome to preserve data in its original language, but for the purposes of grading you should provide english translations. You are welcome to provide multiple visuals in different languages if you would like!

As this assignment is for the whole course, you must incorporate principles discussed in the first week, such as having as high data-ink ratio (Tufte) and aligning with Cairo’s principles of truth, beauty, function, and insight.

Here are the assignment instructions:

 * State the region and the domain category that your data sets are about (e.g., **Ann Arbor, Michigan, United States** and **weather phenomena**).
 * You must state a question about the domain category and region that you identified as being interesting.
 * You must provide at least two links to available datasets. These could be links to files such as CSV or Excel files, or links to websites which might have data in tabular form, such as Wikipedia pages.
 * You must upload an image which addresses the research question you stated. In addition to addressing the question, this visual should follow Cairo's principles of truthfulness, functionality, beauty, and insightfulness.
 * You must contribute a short (1-2 paragraph) written justification of how your visualization addresses your stated research question.

What do we mean by **weather phenomena**?  For this category you might want to consider seasonal changes, natural disasters, or historical trends.

## Tips
* Wikipedia is an excellent source of data, and I strongly encourage you to explore it for new data sources.
* Many governments run open data initiatives at the city, region, and country levels, and these are wonderful resources for localized data sources.
* Several international agencies, such as the [United Nations](http://data.un.org/), the [World Bank](http://data.worldbank.org/), the [Global Open Data Index](http://index.okfn.org/place/) are other great places to look for data.
* This assignment requires you to convert and clean datafiles. Check out the discussion forums for tips on how to do this from various sources, and share your successes with your fellow students!

## Example
Looking for an example? Here's what our course assistant put together for the **Ann Arbor, MI, USA** area using **sports and athletics** as the topic. [Example Solution File](./readonly/Assignment4_example.pdf)

In [2]:
# https://data.worldbank.org/indicator/SP.POP.TOTL?locations=OE&view=chart
# https://covid19.who.int/table
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

EUCountries = {"Countries" :["Austria", "Belgium", "Bulgaria", "Czech Republic", "France", "Germany", "Denmark", "Greece",
               "Estonia", "Ireland", "Spain", "Italy", "Netherlands", "Croatia", "Cyprus", "Latvia", "Lithuania", "Luxembourg",
               "Malta", "Hungary", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia", "Sweden", "Finland",
               "United Kingdom", "Iceland", "Norway", "Liechtenstein", "Switzerland"]}

# Read and prepare Population Data Frame
Population = pd.read_csv("data/PopulationPerCountry.csv")
Population = Population[["Country Name", "Country Code", "2019"]]
Population.rename(columns = {"Country Name":"Country", "Country Code" : "Code", "2019": "Pop2019"}, inplace = True)
Population.replace({"Slovak Republic" : "Slovakia", "United States" : "USA"}, inplace=True)

# Read and prepare Covid Data Frame
Covid = pd.read_csv("data/WHO-COVID-19-global-data.csv")
Covid.rename(columns={"Date_reported" : "Date", " Country_code" : "Code", " Country": "Country",
                      " WHO_region" : "Region", " New_cases": "NewCases", " Cumulative_cases": "CumulativeCases",
                      " New_deaths" : "NewDeaths", " Cumulative_deaths" :"CumulativeDeaths"}, inplace=True)
Covid.replace({"Czechia" : "Czech Republic", "The United Kingdom": "United Kingdom", "United States of America" : "USA"}, inplace=True)

# Checking that country names are the same
# EUdf = pd.DataFrame(EUCountries)
# df1 = pd.merge(EUdf, Population, how='inner', left_on='Countries', right_on='Country')
# df1.to_csv("data/temporary.csv")
# df2 = pd.merge(EUdf, Covid, how='inner', left_on='Countries', right_on='Country')
# df2.to_csv("data/temporary.csv")

# create main DF with all the countries and the Death Rates
df = pd.merge(Covid, Population, how='inner', left_on='Country', right_on='Country')
df["DeathRate"] = df.CumulativeDeaths * 1000000 / df.Pop2019

# Death Rate for Europe
EUTotalPop = Population["Pop2019"][Population.Country.isin(EUCountries["Countries"])].sum()
dfEU = df[df.Country.isin(EUCountries["Countries"])]
dfEU = dfEU.groupby("Date").sum()
dfEU["Europe"] = dfEU["CumulativeDeaths"] * 1000000 / EUTotalPop
dfEU.reset_index(inplace = True)
dfEU = dfEU[["Date", "Europe"]]

# Death Rate for Greece and USA
dfGreece = df[["Date", "DeathRate"]][df.Country == "Greece"]
dfUSA = df[df.Country == "USA"]
dfUSA = df[["Date", "DeathRate"]][df.Country == "USA"]

# Merge fra me to a final DF with index=Date and Columns=Europe, USA, Greece
dfRes = pd.merge(dfEU, dfGreece, how='inner', left_on='Date', right_on='Date')
dfRes.rename(columns = {"DeathRate":"Greece"}, inplace = True)
dfRes = pd.merge(dfRes, dfUSA, how='inner', left_on='Date', right_on='Date')
dfRes.rename(columns = {"DeathRate":"USA"}, inplace = True)
dfRes['Date'] = pd.to_datetime(dfRes['Date'])
dfRes.set_index("Date", inplace=True)

plt.style.use("seaborn-colorblind")
dfRes.plot()
plt.gca().set_title("COVID-19 - Cumulative Deaths growth\nComparison between USA, Europe and Greece")
plt.gca().set_ylabel("Cumulative Deaths per mil of population" )
plt.show()

FileNotFoundError: File b'data/PopulationPerCountry.csv' does not exist